# Imports

In [1]:

import requests
import sqlite3
import pandas as pd
from sodapy import Socrata

# Collect Data

In [14]:
# api endpoint
api_url = "https://data.colorado.gov/resource/4ykn-tg5h.json"
app_token = "0NT6cCdGg6VVAj2pRLjmAqDFu"
username = "glissmann.mark@gmail.com"
password = "K2PRH@mUDwgbfsV"
entityid = "18881009142"
api_key = "gxc0jsptwdopcqxffpl21c8b"
api_secret_key = "2w9yp2sqyp3d7nxkidj8v65e6ow7mos3e7bnuc64rqaiccyc4c" 


In [15]:
batch_size = 1000 
offset = 0

In [2]:
# connect to SQLite database (or create it)
conn = sqlite3.connect('business_entities.db')
cursor = conn.cursor()

In [17]:
# Fetch one record to determine the structure of the data
response = requests.get(api_url, params={'$limit': 1})
if response.status_code == 200:
    sample_data = response.json()
    if sample_data:
        columns = sample_data[0].keys()
        # Create table based on the keys in the data
        cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS colorado (
            {', '.join([f'{col} TEXT' for col in columns])}
        )
        ''')
        conn.commit()
else:
    print(f"Failed to retrieve data: {response.status_code}")
    exit()

In [18]:
while True:
    # fetch a batch of data
    params = {
        '$limit': batch_size,
        '$offset': offset
    }

    response = requests.get(api_url, params=params)

    if response.status_code == 200:
        data = response.json()

        if not data:
            # no more data to fetch
            break
        
        # prepare insert statement
        placeholders = ', '.join(['?'] * len(columns))
        insert_statement = f'''
        INSERT OR IGNORE INTO colorado ({', '.join(columns)})
        VALUES ({placeholders})
        '''

        # process and insert data into the database
        for item in data:
            values = tuple(item.get(col) for col in columns)
            cursor.execute(insert_statement, values)
        
        # commit changes to the database
        conn.commit()

        # increment offset for the next batch
        offset += batch_size
    else:
        print(f"Failed to retreive data: {response.status_code}")
        break 

# close the database connection
conn.close()

# Cleanse Data

In [8]:
cleanse_queries = []

cleanse_queries.append("""
-- Step 1: Fill missing principalcountry values with 'US'
UPDATE colorado
SET principalcountry = 'US'
WHERE principalcountry IS NULL;
""")

cleanse_queries.append("""
-- Step 2: Standardize capitalization
UPDATE colorado
SET 
    agentprincipaladdress1 = UPPER(agentprincipaladdress1),
    agentprincipalcity = UPPER(agentprincipalcity),
    agentprincipalcountry = UPPER(agentprincipalcountry),
    entityname = UPPER(entityname),
    principaladdress1 = UPPER(principaladdress1),
    principalcity = UPPER(principalcity),
    principalcountry = UPPER(principalcountry);
""")
                       
cleanse_queries.append("""
-- Step 3: Extract date from timestamp
UPDATE colorado
SET entityformdate = DATE(entityformdate);
""")
                       
cleanse_queries.append("""
-- Step 4: Create a new table without columns with majority null values
CREATE TABLE colorado_new AS
SELECT
    entityid,
    entityname,
    principaladdress1,
    principalcity,
    principalstate,
    principalzipcode,
    principalcountry,
    entitystatus,
    jurisdictonofformation,
    entitytype,
    agentfirstname,
    agentlastname,
    agentprincipaladdress1,
    agentprincipalcity,
    agentprincipalstate,
    agentprincipalzipcode,
    agentprincipalcountry,
    entityformdate
FROM colorado;
""")
                       
cleanse_queries.append("""
-- Drop the old table and rename the new table
DROP TABLE colorado;
""")
                       
cleanse_queries.append("""
ALTER TABLE colorado_new RENAME TO colorado;
""")


In [25]:
cursor.execute("VACUUM")

OperationalError: database or disk is full

In [24]:
for query in cleanse_queries:
    print(f"Executing query: {query}")
    cursor.execute(query)


Executing query: 
-- Step 1: Fill missing principalcountry values with 'US'
UPDATE colorado
SET principalcountry = 'US'
WHERE principalcountry IS NULL;

Executing query: 
-- Step 2: Standardize capitalization
UPDATE colorado
SET 
    agentprincipaladdress1 = UPPER(agentprincipaladdress1),
    agentprincipalcity = UPPER(agentprincipalcity),
    agentprincipalcountry = UPPER(agentprincipalcountry),
    entityname = UPPER(entityname),
    principaladdress1 = UPPER(principaladdress1),
    principalcity = UPPER(principalcity),
    principalcountry = UPPER(principalcountry);



OperationalError: database or disk is full

# Query Data

In [26]:
# connect to SQLite database (or create it)
conn = sqlite3.connect('business_entities.db')
cursor = conn.cursor()

In [30]:
# Execute a SQL query
test_query = """
SELECT 
    * 
FROM colorado;
"""
cursor.execute(test_query)

# Fetch all results from the executed query
results = cursor.fetchall()

# Optionally, load the results into a pandas DataFrame for easier data manipulation
df = pd.DataFrame(results, columns=[col[0] for col in cursor.description])

# Print the DataFrame
#print(df)

# Write file
df.to_csv('business_entities', index=False)

In [4]:
# close the database connection
conn.close()